In [1]:
import multiprocessing as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
basedir = './국토교통부_건축물대장_층별개요/'
filenames = [f for f in os.listdir(basedir) if f.endswith('.txt')]
filenames

['DJY_MART_03_201412.txt',
 'mart_djy_04_201612.txt',
 'mart_djy_04_201712.txt',
 'mart_djy_04_201812.txt',
 'mart_djy_04_201912.txt',
 'mart_djy_04_202011.txt',
 'MART_DJY_FLR_OULN_201512.txt']

In [3]:
colnames_df = pd.read_excel('./국토교통부_건축물대장_층별개요/데이터구조.xls', header=1)
colnames = colnames_df['컬럼 한글명'].tolist()
colnames

['관리_건축물대장_PK',
 '대지_위치',
 '도로명_대지_위치',
 '건물_명',
 '시군구_코드',
 '법정동_코드',
 '대지_구분_코드',
 '번',
 '지',
 '특수지_명',
 '블록',
 '로트',
 '새주소_도로_코드',
 '새주소_법정동_코드',
 '새주소_지상지하_코드',
 '새주소_본_번',
 '새주소_부_번',
 '동_명',
 '층_구분_코드',
 '층_구분_코드_명',
 '층_번호',
 '층_번호_명',
 '구조_코드',
 '구조_코드_명',
 '기타_구조',
 '주_용도_코드',
 '주_용도_코드_명',
 '기타_용도',
 '면적(㎡)',
 '주_부속_구분_코드',
 '주_부속_구분_코드_명',
 '면적_제외_여부',
 '생성_일자']

In [4]:
for file in filenames:
    print(file)
    f = open(basedir + file, 'r')
    print(f.readline())
    f.close()

DJY_MART_03_201412.txt
마트_층별_개요_PK,건축물대장_PK,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블럭,로트,지번_좌표_X,지번_좌표_Y,도로명_좌표_X,도로명_좌표_Y,기타_기재_사항,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명칭,층_구분_코드,층_구분_코드_명,층_번호,층_번호_명,구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,면적,주_부속_구분_코드,주_부속_구분_코드_명,면적_제외_여부,생성_일자

mart_djy_04_201612.txt
26530-100175951|부산광역시 사상구 주례동 67-9번지|부산광역시 사상구 주례로9번길 11||26530|10600|0|0067|0009||||265304217356|10601|0|11|0|주건축물제1동|20|지상|1|1층|21|철근콘크리트구조|철근콘크리트구조|04402|사무소|제2종근린생활시설(사무소(주차장)|23.04|0|주건축물|0|20100105

mart_djy_04_201712.txt
ORACLEDUMP	|MGM_BLDRGST_PK	PLAT_PLC	NEW_PLAT_PLC	BLD_NM	SIGUNGU_CD	BJDONG_CD	PLAT_GB_CD	BUN	JI	SPLOT_NM	BLOCK	LOT	NA_ROAD_CD	NA_BJDONG_CD	NA_UGRND_CD	NA_MAIN_BUN	NA_SUB_BUN	DONG_NM	FLR_GB_CD	FLR_GB_CD_NM	FLR_NO	FLR_NO_NM	STRCT_CD	STRCT_CD_NM	ETC_STRCT	MAIN_PURPS_CD	MAIN_PURPS_CD_NM	ETC_PURPS	AREA	MAIN_ATCH_GB_CD	MAIN_ATCH_GB_CD_NM	AREA_EXCT_YN	CRTN_DAY

mart_djy_04_201812.txt
29155-8546|광주광역시 남구 양촌동 358번지| 광주광역시 남구 포충로 125-5||2

In [4]:
floor_info_2014_df = pd.read_csv(basedir + 'DJY_MART_03_201412.txt', encoding='euc-kr')
print(floor_info_2014_df.shape)
floor_info_2014_df.head()

c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (10,11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(3663172, 39)


,마트_층별_개요_PK,건축물대장_PK,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블럭,로트,지번_좌표_X,지번_좌표_Y,도로명_좌표_X,도로명_좌표_Y,기타_기재_사항,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명칭,층_구분_코드,층_구분_코드_명,층_번호,층_번호_명,구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,면적,주_부속_구분_코드,주_부속_구분_코드_명,면적_제외_여부,생성_일자
0,41465-1290,41465-10023034802,경기도 용인시 수지구 성복동 759번지,NaN,버들치마을 성복 힐스테이트 2차,41465,10600,0,759,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.146544e+11,10601.0,0,13.0,0.0,223,20.0,지상,1,1층,21.0,철근콘크리트구조,철근콘크리트구조,02001,아파트,경로당,216.785,1.0,부속건축물,0.0,20141109
1,41465-1291,41465-10023034902,경기도 용인시 수지구 성복동 759번지,NaN,버들치마을 성복 힐스테이트 2차,41465,10600,0,759,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.146544e+11,10601.0,0,13.0,0.0,224,20.0,지상,1,1층,21.0,철근콘크리트구조,철근콘크리트구조,02001,아파트,문주,47.947,1.0,부속건축물,1.0,20141109
2,41610-1292,41610-10018964202,경기도 광주시 장지동 599-11번지,NaN,NaN,41610,11100,0,599,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-건축-신축허가-5,4.161044e+11,11101.0,0,59.0,0.0,NaN,20.0,지상,4,4층,21.0,철근콘크리트구조,철근콘크리트구조,02003,다세대주택,다세대주택,129.120,0.0,주건축물,0.0,20141109
3,41610-1293,41610-10018964202,경기도 광주시 장지동 599-11번지,NaN,NaN,41610,11100,0,599,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-건축-신축허가-5,4.161044e+11,11101.0,0,59.0,0.0,NaN,20.0,지상,3,3층,21.0,철근콘크리트구조,철근콘크리트구조,02003,다세대주택,다세대주택,129.120,0.0,주건축물,0.0,20141109
4,41610-1294,41610-10018964202,경기도 광주시 장지동 599-11번지,NaN,NaN,41610,11100,0,599,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-건축-신축허가-5,4.161044e+11,11101.0,0,59.0,0.0,NaN,30.0,옥탑,1,옥탑1층,21.0,철근콘크리트구조,철근콘크리트구조,02003,다세대주택,계단실,10.080,0.0,주건축물,1.0,20141109


In [5]:
floor_info_2014_df['구조_코드_명'].value_counts()

철근콘크리트구조        3353230
벽돌구조             200253
철골철근콘크리트구조        31948
기타조적구조            20387
철골콘크리트구조          16470
일반철골구조            13089
프리케스트콘크리트구조        7745
블록구조               7475
경량철골구조             5579
기타콘크리트구조           1824
일반목구조              1739
기타구조                342
기타강구조               295
콘크리트구조              207
강파이프구조              171
기타철골철근콘크리트구조         69
조적구조                 43
통나무구조                20
석구조                   6
철골구조                  2
Name: 구조_코드_명, dtype: int64

In [6]:
floor_info_2014_df['구조_코드'].value_counts()

21.0    3353230
11.0     200253
42.0      31948
19.0      20387
41.0      16470
31.0      13089
22.0       7745
12.0       7475
32.0       5579
29.0       1824
51.0       1739
99.0        342
39.0        295
20.0        207
33.0        171
49.0         69
10.0         43
52.0         20
2.0          19
3.0           6
13.0          6
35.0          5
30.0          2
28.0          2
27.0          1
38.0          1
1.0           1
Name: 구조_코드, dtype: int64

In [7]:
floor_info_2014_df['구조_코드_명'].nunique()

20

In [8]:
floor_info_2014_df['구조_코드'].nunique()

27

In [8]:
floor_info_2014_df.dropna(subset=['대지_위치'], inplace=True)

In [9]:
seoul_2014_df = floor_info_2014_df[floor_info_2014_df['대지_위치'].str.contains('서울특별시')]
print(seoul_2014_df.shape)

(956287, 39)


In [11]:
seoul_2014_df['기준년월'] = 201412

In [12]:
seoul_2014_df.to_csv(basedir + 'floor_info_baseline_seoul_201412.csv', index=False)

In [13]:
floor_info_df = pd.read_csv(basedir + 'mart_djy_04_201712.txt', sep='|', encoding='euc-kr', header=0, names=colnames)

c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (10,11,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
floor_info_df.shape

(19523850, 33)

In [15]:
floor_info_df.head()

,관리_건축물대장_PK,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,층_구분_코드,층_구분_코드_명,층_번호,층_번호_명,구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,면적(㎡),주_부속_구분_코드,주_부속_구분_코드_명,면적_제외_여부,생성_일자
0,29155-14451,광주광역시 남구 주월동 974-81번지,광주광역시 남구 군분로23번길 8-1,NaN,29155,10600.0,0.0,974,81,NaN,NaN,NaN,2.915543e+11,10601.0,0,8.0,1.0,NaN,20.0,지상,1,1층,11,벽돌구조,시멘트벽돌,01003,다가구주택,1가구 주택,101.89,0.0,주건축물,NaN,20100105
1,29155-14451,광주광역시 남구 주월동 974-81번지,광주광역시 남구 군분로23번길 8-1,NaN,29155,10600.0,0.0,974,81,NaN,NaN,NaN,2.915543e+11,10601.0,0,8.0,1.0,NaN,20.0,지상,2,2층,11,벽돌구조,시멘트벽돌,01003,다가구주택,2가구 주택,92.97,0.0,주건축물,NaN,20100105
2,29155-8546,광주광역시 남구 양촌동 358번지,광주광역시 남구 포충로 125-5,NaN,29155,11700.0,0.0,358,0,NaN,NaN,NaN,2.915532e+11,11701.0,0,125.0,5.0,NaN,20.0,지상,1,1층,11,벽돌구조,세멘벽돌,01001,단독주택,주택,59.72,0.0,주건축물,NaN,20100105
3,29155-40705,광주광역시 남구 주월동 997-7번지,광주광역시 남구 군분로77번안길 29-2,NaN,29155,10600.0,0.0,997,7,NaN,NaN,NaN,2.915543e+11,10602.0,0,29.0,2.0,NaN,20.0,지상,2,2층,11,벽돌구조,세벽,01001,단독주택,주택,52.24,0.0,주건축물,NaN,20100105
4,29155-40705,광주광역시 남구 주월동 997-7번지,광주광역시 남구 군분로77번안길 29-2,NaN,29155,10600.0,0.0,997,7,NaN,NaN,NaN,2.915543e+11,10602.0,0,29.0,2.0,NaN,20.0,지상,1,1층,11,벽돌구조,세벽,01001,단독주택,"주택,점포",78.11,0.0,주건축물,NaN,20100105


In [16]:
floor_info_df.dropna(subset=['대지_위치'], inplace=True)

In [18]:
seoul_df = floor_info_df[floor_info_df['대지_위치'].str.contains('서울특별시')]
print(seoul_df.shape)

(2952425, 33)


In [19]:
seoul_df['기준년월'] = 201712
seoul_df.to_csv(basedir + 'floor_info_baseline_seoul_201712.csv', index=False)

In [5]:
no_header_files_list = ['mart_djy_04_201612.txt', 'MART_DJY_FLR_OULN_201512.txt', 'mart_djy_04_201812.txt',
                        'mart_djy_04_201912.txt', 'mart_djy_04_202011.txt']

In [20]:
for file in tqdm(no_header_files_list):
    yearmonth = file.replace('.txt', '')[-6:]
    year = yearmonth[:4]
    yearmonth = int(yearmonth)
    
    try:
        df = pd.read_csv(basedir + file, encoding='euc-kr', sep='|', header=None, names=colnames)
    except:
        try:
            df = pd.read_csv(basedir + file, sep='|', header=None, names=colnames)
        except:
            raise ValueError(file)
    df.dropna(subset=['대지_위치'], inplace=True)
    seoul_df = df[df['대지_위치'].str.contains('서울특별시')]
    seoul_df['기준년월'] = yearmonth
    seoul_df.to_csv(basedir + 'floor_info_baseline_seoul_{}.csv'.format(yearmonth), index=False)

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (10,11,22,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
 20%|████████████████▌                                                                  | 1/5 [01:50<07:22, 110.55s/it]c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (2,9,10,11,22,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
 60%|█████████████████████████████████████████████████▊                                 | 3/5 [05:40<03:49, 114.60s/it]c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (11,22) have mixed types. Specify dt